In [1]:
### METODO PARA TRADUCIR 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import LanguageTranslatorV3

authenticator = IAMAuthenticator('In3r0-dDlor3CtpyZNIZGk7gzgradLVUYU9lS7EknPGU')
language_translator = LanguageTranslatorV3(
    version='2018-05-01',
    authenticator=authenticator
)

language_translator.set_service_url('https://api.us-south.language-translator.watson.cloud.ibm.com/instances/9472bba7-295e-45cf-98d3-8582c07615e6')


In [2]:
### NATURAL LANGUAGE UNDERSTANDING
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions,EmotionOptions, KeywordsOptions,MetadataOptions, RelationsOptions, CategoriesOptions, ConceptsOptions, SemanticRolesOptions, SentimentOptions, SyntaxOptions, SyntaxOptionsTokens
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('r7Wm41T8suQL4T4JEDASW3g7iJxEhe-jpTB84KMdBl1A')
service = NaturalLanguageUnderstandingV1(
     version='2018-03-16',
     authenticator=authenticator)
service.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/c54f98e9-4089-4a5b-8ea8-29f7cb6f786f')

In [3]:
def tipoLenguaje(mensaje):
    language = language_translator.identify(
        mensaje).get_result()
    r= json.dumps(language, indent=2)
    rp= json.loads(r)
    return rp["languages"][0]["language"]

def metodoTraducir(mensaje):
    tam=tipoLenguaje(mensaje)
    if(tam == "fr" or tam=="ca" or tam=="en"):
        print("Idioma: ",tam)
        translation = language_translator.translate(
        text=mensaje,
        model_id=tam+"-es").get_result()
        r= json.dumps(translation, indent=2)
        rp= json.loads(r)
        return rp["translations"][0]["translation"],tam
    else:
        return mensaje,"es"

def traducirTexto(mensaje,tam):
    if(tam =="fr" or tam=="ca" or tam=="en"):
        translation = language_translator.translate(
        text=mensaje,
        model_id="es-"+tam).get_result()
        r= json.dumps(translation, indent=2)
        rp= json.loads(r)
        return rp["translations"][0]["translation"]
    else:
        return mensaje

In [ ]:
#Importar librerias
#from ibm_cloud_sdk_core.api_exception import ApiException
import json
import requests

import random
from flask import Flask, request
from pymessenger.bot import Bot


import json
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import mysql.connector
from clips import Environment, Symbol
    
import mysql.connector

from clips import Environment, Symbol

import re
#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
from neo4j import GraphDatabase

uri = "bolt://35.184.105.166:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "admin"),encrypted=False)

result=[]
parques = []
centralidad = []
def get_mo(tx):
    result = tx.run("CALL gds.alpha.closeness.stream({nodeProjection: 'Plato', "
                    "relationshipProjection: 'LINK' })"
                    "YIELD nodeId, centrality "
                    "RETURN gds.util.asNode(nodeId).name AS user, centrality "
                    "ORDER BY centrality DESC")
    for record in result:
        #print(record)
        parques.append(record["user"])
        centralidad.append(record["centrality"])

def recibir():
    with driver.session() as session:
        ciudades = session.read_transaction(get_mo)
        #print(ciudades)
        #print("Solución")
        #print("Nodo -> Centralidad")
        for i in range(len(parques)):
            result=parques[0]
            #print(parques[0])
            #print(parques[1], " -> ", centralidad[1])
            #print(parques[2], " -> ", centralidad[2])
    #driver.close()
    #print(result)
    return result

#print(recibir())
#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

uri = "bolt://35.184.105.166:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "admin"),encrypted=False)

resultado=[]
parques = []
centralidad = []
def get_mo(tx):
    result = tx.run("CALL gds.louvain.stream({nodeProjection: 'Plato', "
                    "relationshipProjection: {"
                    "TYPE:{"
                    "type: 'LINK', "
                    "orientation: 'undirected',"
                    "aggregation: 'NONE'"
                    "}"
                    "},"
                    "includeIntermediateCommunities: true"
                    "})YIELD nodeId, communityId, intermediateCommunityIds "
                    "RETURN gds.util.asNode(nodeId).name AS name, communityId, intermediateCommunityIds")
    for record in result:
        #print(record)
        parques.append(record["name"])
        centralidad.append(record["communityId"])

def recibirL():
    with driver.session() as session:
        ciudades = session.read_transaction(get_mo)
        #print("Solución")
        #print("Nodo -> Centralidad")
        for i in range(len(parques)):
            #resultado=parques[i]
            resultado=parques[7]
            #print(parques[i], " -> ", centralidad[i])    
    #print(resultado)        
    #driver.close()
    return resultado

#print(recibirL())
#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
 
#Variables para el Token y la URL del chatbot
TOKEN = "1379923032:AAHh-rNcP67IiJZ0KrrQ6CUeaoLiKlzoJ1A" #Cambialo por tu token
URL = "https://api.telegram.org/bot" + TOKEN + "/"
 
 
 
def Find(string): 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)     
    for x in url:
        return x[0]
    
    
def update(offset):
    #Llamar al metodo getUpdates del bot, utilizando un offset
    respuesta = requests.get(URL + "getUpdates" + "?offset=" + str(offset))
    #Telegram devolvera todos los mensajes con id IGUAL o SUPERIOR al offset
 
 
    #Decodificar la respuesta recibida a formato UTF8
    mensajes_js = respuesta.content.decode("utf8")
 
    #Convertir el string de JSON a un diccionario de Python
    mensajes_diccionario = json.loads(mensajes_js)
 
    #Devolver este diccionario
    return mensajes_diccionario
 
def leer_mensaje(mensaje):
 
    #Extraer el texto, nombre de la persona e id del último mensaje recibido
    texto = mensaje["message"]["text"]
    persona = mensaje["message"]["from"]["first_name"]
    id_chat = mensaje["message"]["chat"]["id"]
 
    #Calcular el identificador unico del mensaje para calcular el offset
    id_update = mensaje["update_id"]
 
    #Devolver las dos id, el nombre y el texto del mensaje
    return id_chat, persona, texto, id_update
 
def enviar_mensaje(idchat, texto):
    #Llamar el metodo sendMessage del bot, passando el texto y la id del chat
    requests.get(URL + "sendMessage?text=" + texto + "&chat_id=" + str(idchat))
##Consumo del Asitente de Watson
def get_message(mensaje):
    res=''
    aux=True
    if(mensaje == "cotizado" or mensaje == "plato mas cotizado" or mensaje == "plato cotizado"):
        #print('holas')
        cercania=len(recibir())
        print(cercania)
        if(cercania == 14):
            aux=False
            #return " El plato mas cotizado es: " + str(recibir()) + " Escriba Receta \"" + str(recibir()) + "\""
            return " El plato mas cotizado es: " + str(recibir())
    if(mensaje == "vendido" or mensaje == "plato mas vendido" or mensaje == "plato vendido"):
        #print('aui')
        louvain=len(recibirL())
        print(louvain)
        if(louvain == 27):
            aux=False
            #return "El plato mas vendido es: " + str(recibirL()) + " Escriba Receta \"" + str(recibirL()) + "\""
            return "El plato mas vendido es: " + str(recibirL())
    print(mensaje)
    lst=mensaje.split(',')
    print(" SOY ARREGLO")
    print(lst)
    verificar=Find(mensaje)
    if(verificar != None):
        response = service.analyze(
        url=verificar,
        features=Features(concepts=ConceptsOptions())).get_result()
        jsonData = json.dumps(response, indent=2)
        jsonToPython = json.loads(jsonData)
        #v = jsonToPython["output"]
        v = jsonToPython["concepts"]
        res1 = v[0]['text']
        print(res1)
        print(json.dumps(response, indent=2))
        aux=False
        return " El concepto mas relevante de la pagina es: " + str(res1)
    print(mensaje)
    mensaje=metodoTraducir(mensaje)
    authenticator = IAMAuthenticator('cZrTUNTebpncIkhsSC0kR-HyNx9cNmKsrtfW_2vyJXPd')
    assistant = AssistantV2(
    version='2020-04-01',
    authenticator = authenticator
    )
    assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com/instances/dcc4a58c-6f8d-4736-a997-41604c06f7a2')
    session = assistant.create_session(
    assistant_id='9d7b4d15-3e32-4abe-a852-c691b36ce04d'
    ).get_result()
    message = assistant.message(
    '9d7b4d15-3e32-4abe-a852-c691b36ce04d',
    session['session_id'],
    input={
        'message_type': 'text',
        'text': "'"+mensaje[0]+"'"
        #'text': "'"+mensaje+"'"
    }
    ).get_result()   
    jsonData = json.dumps(message, indent=2)
    jsonToPython = json.loads(jsonData)
    v = jsonToPython["output"]
    res = v['generic'][0]['text']
    #return res
    return traducirTexto(res,mensaje[1])
    
##Variable para almacenar la ID del ultimo mensaje procesado
ultima_id = 0
 
while(True):
    mensajes_diccionario = update(ultima_id)
    for i in mensajes_diccionario["result"]:
 
        #Llamar a la funcion "leer_mensaje()"
        idchat, nombre, texto, id_update = leer_mensaje(i)
 
        #Si la ID del mensaje es mayor que el ultimo, se guarda la ID + 1
        if id_update > (ultima_id-1):
            ultima_id = id_update + 1
 
        #Generar una respuesta a partir de la informacion del mensaje
        #if "Hola" in texto:
            #texto_respuesta = "Hola, " + nombre + "!"
        #elif "Adios" in texto:
            #texto_respuesta = "Hasta pronto!"
        #else:
            #texto_respuesta = "Has escrito: \"" + texto + "\""
            texto_respuesta = get_message(texto)
 
        #Enviar la respuesta
        enviar_mensaje(idchat, texto_respuesta)
 
    #Vaciar el diccionario
    mensajes_diccionario = []

menu
 SOY ARREGLO
['menu']
menu
hola
 SOY ARREGLO
['hola']
hola
mas recetas
 SOY ARREGLO
['mas recetas']
mas recetas
recomendacion
 SOY ARREGLO
['recomendacion']
recomendacion
14
1.
 SOY ARREGLO
['1.']
1.
menu
 SOY ARREGLO
['menu']
menu
hello
 SOY ARREGLO
['hello']
hello
Idioma:  en
ceviche de prawn
 SOY ARREGLO
['ceviche de prawn']
ceviche de prawn
Idioma:  fr
menu
 SOY ARREGLO
['menu']
menu
mas recetas
 SOY ARREGLO
['mas recetas']
mas recetas
recomendacion
 SOY ARREGLO
['recomendacion']
recomendacion
14
1.
 SOY ARREGLO
['1.']
1.
27
8.
 SOY ARREGLO
['8.']
8.
analizar pagina
 SOY ARREGLO
['analizar pagina']
analizar pagina
https://www.yachaytech.edu.ec/
 SOY ARREGLO
['https://www.yachaytech.edu.ec/']
Aseguramiento de la calidad
{
  "usage": {
    "text_units": 1,
    "text_characters": 619,
    "features": 1
  },
  "retrieved_url": "https://www.yachaytech.edu.ec/",
  "language": "es",
  "concepts": [
    {
      "text": "Aseguramiento de la calidad",
      "relevance": 0.897894,
      